In [1]:
! pip install requests_html

    100% |████████████████████████████████| 1.2MB 7.9MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 14.5MB/s a 0:00:01
    100% |████████████████████████████████| 71kB 12.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 13.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for fake-useragent ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Running setup.py bdist_wheel for pyppeteer ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
  Running setup.py bdist_wheel for parse ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/16/f1/d5/72f7e2c22ab3d106bb6f7d727552b5e027b832284f4704b106
Successfully built fake-useragent bs4 pyppeteer parse


In [4]:
from requests_html import HTMLSession
session = HTMLSession()
f = open("fpt.txt", "w")
for i in range(20):
    web = session.get("https://fptshop.com.vn/dien-thoai?sort=ban-chay-nhat?trang={}".format(i + 1))
    list = web.html.find("div.fs-ctf-cr", first = True).find("div.fs-lpil")
    url = [[x, x.find('a')] for x in list]
    url_x = [[x[1][0].attrs["href"], x[0].find('img', first=True).attrs["data-original"]] for x in url]
    for x in url_x:
        f.write(x[0] + " " + x[1] + '\n')
        print(x)
f.close()

['/dien-thoai/samsung-galaxy-fold', 'https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/11/25/637102754985154642_samsung-galaxy-fold-den-dd.png']
['/dien-thoai/iphone-11-pro-max-512gb', 'https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/637037687757921048_11-pro-max-chung.png']
['/dien-thoai/iphone-11-pro-512gb', 'https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/637037672773372121_11-pro-chung.png']
['/dien-thoai/iphone-11-pro-max-256gb', 'https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/637037687757921048_11-pro-max-chung.png']
['/dien-thoai/iphone-11-pro-256gb', 'https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/6370376

In [5]:
from requests_html import HTMLSession
import time
import codecs

def getCompany(url):
    return url[34:].split('-')[0]
def getNumber(s):
    if (s.find('.') >= 0):
        s = s[:s.find('.')]
    a = [x for x in s if '0' <= x <= '9']
    return ''.join(a)
def getCamera(s):
    s = s.lower()
    s = s[:s.find('mp')].strip()
    return getNumber(s)
def getBattery(s):
    if s.find('mAh') >= 0:
        return s[:s.find('mAh')].strip()
    return s
def getScreenSize(s):
    ps = ["inchs", "inches", "\""]
    for p in ps:
        x = s.find(p)
        if x >= 0:
            return s[:x].strip()
    return s
def getCPU(s):
    return s.split(',')[0].strip()
def process_fpt(url):
    web = session.get(url)
    res = {'prices': "", 'screens': "", 'os': "", 'front_cams': "", 'back_cams': "", 'cpus': "",'rams': "", 'memories': "",'batteries': "", 'urls': "", 'companies': ""}
    res['prices'] = web.html.find('p.fs-dtprice', first=True).text.split(' ')[0]
    info = web.html.find("div.fs-tsright", first=True).find("li")
    res["urls"] = url
    res['companies'] = getCompany(url)
    for x in info:
        key = x.find("label", first=True).text
        if (key == "Màn hình :"):
            res['screens'] = getScreenSize(x.find("span", first=True).text)
        elif key == "Camera trước :":
            res['front_cams'] = getCamera(x.find("span", first=True).text)
        elif key == "Camera sau :":
            res['back_cams'] = getCamera(x.find("span", first=True).text)
        elif key == "RAM :":
            res['rams'] = getNumber(x.find("span", first=True).text)
        elif key == "Bộ nhớ trong :":
            res['memories'] = getNumber(x.find("span", first=True).text)
        elif key == "CPU :":
            res['cpus'] = getCPU(x.find("span", first=True).text)
        elif key == "Dung lượng pin :":
            res['batteries'] = getBattery(x.find("span", first=True).text)
        elif key == "Hệ điều hành :":
            res['os'] = x.find("span", first=True).text.lower()
                                                                         
                                                                         
    return res
f= open("fpt.txt","r")
lines = f.readlines()
g = codecs.open("fpt.csv", "w","utf-8")
g.write(u"images	prices	screens	os	front_cams	back_cams	cpus	rams	memories	batteries	urls	companies\n")
for line in lines:
    line = line.split(' ')
    x = process_fpt("https://fptshop.com.vn" + line[0])
    x["images"] = line[1]
    g.write(u"{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(x["images"], x["prices"],x["screens"], x["os"],x["front_cams"], x["back_cams"],x["cpus"], x["rams"],x["memories"], x["batteries"],x["urls"], x["companies"]))
    print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t".format(x["images"], x["prices"],x["screens"], x["os"],x["front_cams"], x["back_cams"],x["cpus"], x["rams"],x["memories"], x["batteries"],x["urls"], x["companies"]))
    time.sleep(1)
f.close()
g.close()


https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/11/25/637102754985154642_samsung-galaxy-fold-den-dd.png
	50.000.000	Chính 7.3, Super AMOLED, Full HD+, 2152 x 1536 Pixels	android 9.0 (pie)	10	12	1 nhân 2.84 GHz	12	512	4380	https://fptshop.com.vn/dien-thoai/samsung-galaxy-fold	samsung	
https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/637037687757921048_11-pro-max-chung.png
	43.990.000	6.5	ios 13	12	12	Apple A13 Bionic	4	512	Lâu hơn iPhone Xs Max 5h	https://fptshop.com.vn/dien-thoai/iphone-11-pro-max-512gb	iphone	
https://images.fpt.shop/unsafe/fit-in/192x192/filters:quality(90):fill(white)/cdn.fptshop.com.vn/Uploads/Originals/2019/9/11/637037672773372121_11-pro-chung.png
	40.990.000	5.8	ios 13	12	12	Apple A13 Bionic	4	512	Lâu hơn iPhone Xs 4h	https://fptshop.com.vn/dien-thoai/iphone-11-pro-512gb	iphone	
https://images.fpt.shop/unsafe/fit-in/1